<a href="https://colab.research.google.com/github/Anshika0309/Anshika/blob/main/PRO_C115_Boilerplate_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load the Dataset

In [40]:
#Load the dataset from given directory
!git clone https://github.com/procodingclass/PRO-C114-Text-Sentiment-Dataset

fatal: destination path 'PRO-C114-Text-Sentiment-Dataset' already exists and is not an empty directory.


## Pandas:
**Pandas** is an open-source library built on top of Numpy and Matplotlib. It provides high-performance, easy-to-use data structures and data analysis tools.

A DataFrame is a two-dimensional data structure, i.e., data is aligned in a tabular fashion in rows and columns. Pandas DataFrame consists of three principal components, the data, rows, and columns.

Pandas DataFrame will be created by loading the datasets from existing MS Excel files, CSV files or SQL Database. Pandas DataFrame can also be created from the lists, dictionaries etc.


## Use Pandas to display the dataset


In [41]:
import pandas as pd

#read excel file
train_data_raw = pd.read_excel("/content/PRO-C114-Text-Sentiment-Dataset/text-emotion-training-dataset.xlsx")

#display first five entries of training dataset
train_data_raw.head()

,Text_Emotion
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger


## Split the rows in two columns as Text and Emotions

In [42]:
#Split the rows in two columns as Text and Emotions
train_data = pd.DataFrame(train_data_raw["Text_Emotion"].str.split(";",1).tolist(),
                                                     columns = ['Text','Emotion'])

<ipython-input-42-3bca6dacc732>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  train_data = pd.DataFrame(train_data_raw["Text_Emotion"].str.split(";",1).tolist(),


In [43]:
train_data.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


## Giving labels to Emotions

In [44]:
#Find unique emotions
train_data["Emotion"].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [45]:
#Create a Dictionary to replace emotions with labels
encode_emotions = {"anger": 0, "fear": 1, "joy": 2, "love": 3, "sadness": 4, "surprise": 5}

In [46]:
train_data.replace(encode_emotions, inplace = True)
train_data.head()

,Text,Emotion
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0


## Convert Dataframe to list of dataset

In [47]:
# Convert Dataframe to list of dataset

training_sentences = []
training_labels = []

# append text and emotions in the list using the 'loc' method

for i in range(len(train_data)):

  sentence = train_data.loc[i, "Text"]
  training_sentences.append(sentence)

  label = train_data.loc[i, "Emotion"]
  training_labels.append(label)

In [48]:
#Check a random text and label of the list

training_sentences[50], training_labels[50]

('i need to feel the dough to make sure its just perfect', 2)

## Tokenization & Padding

The act of converting text into numbers is known as **Tokenization**. The **Tokenizer** class of Keras is used for encoding text input into integer sequence.

In [49]:
#import Tokenizer from tensorflow

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

#Define parameters for Tokenizer

vocab_size = 10000
embedding_dim = 16
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

#Create a word_index dictionary

word_index = tokenizer.word_index

#Check numeral value assigned to a word

word_index["the"]

6

In [50]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)

training_sequences[0:2]

[[2, 139, 3, 679],
 [2,
  40,
  101,
  60,
  8,
  15,
  494,
  5,
  15,
  3496,
  553,
  32,
  60,
  61,
  128,
  148,
  76,
  1480,
  4,
  22,
  1255]]

**Padding** It is important to make all the sentences contain the same number of words. Zero is used for padding the tokenized sequence to make text contain the same number of tokens.

In [51]:
#Define parameters for pad_sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_type='post'
max_length = 100
trunc_type='post'


training_padded = pad_sequences(training_sequences, maxlen=max_length,
                                padding=padding_type, truncating=trunc_type)

training_padded

array([[   2,  139,    3, ...,    0,    0,    0],
       [   2,   40,  101, ...,    0,    0,    0],
       [  17, 3060,    7, ...,    0,    0,    0],
       ...,
       [   2,    3,  327, ...,    0,    0,    0],
       [   2,    3,   14, ...,    0,    0,    0],
       [   2,   47,    7, ...,    0,    0,    0]], dtype=int32)

## Converting padded sequences and labels into an Numpy array


In [52]:
#Create numpy arrays for padded sequences and labels
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

## Model Compilation

In [53]:
#Define different layers and Compile model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.layers import Conv1D,Dropout,MaxPooling1D
model = tf.keras.Sequential([Embedding(vocab_size,embedding_dim,input_length = max_length),Dropout(0.2),
                             Conv1D(filters = 256,kernel_size = 3,activation = "relu"),
                             MaxPooling1D(pool_size = 3),
                             Conv1D(filters = 128,kernel_size = 3,activation = "relu"),
                             MaxPooling1D(pool_size = 3),
                             LSTM(128),
                             Dense(128,activation = "relu"),
                             Dropout(0.2),
                             Dense(64,activation = "relu"),
                             Dense(6,activation = "softmax")])


## Model Summary

In [57]:
#use the summary() method
model.compile(loss = "sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [58]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 16)           160000    
                                                                 
 dropout_6 (Dropout)         (None, 100, 16)           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 98, 256)           12544     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 32, 256)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 30, 128)           98432     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 10, 128)          0         
 1D)                                                  

## Train Model

In [59]:
#use the fit() function for training model
number_epochs = 30
history = model.fit(training_padded,training_labels,epochs=number_epochs,verbose=2)

Epoch 1/30
500/500 - 33s - loss: 1.4510 - accuracy: 0.3489 - 33s/epoch - 66ms/step
Epoch 2/30
500/500 - 38s - loss: 0.8630 - accuracy: 0.6428 - 38s/epoch - 76ms/step
Epoch 3/30
500/500 - 31s - loss: 0.5383 - accuracy: 0.8148 - 31s/epoch - 62ms/step
Epoch 4/30
500/500 - 32s - loss: 0.4312 - accuracy: 0.8602 - 32s/epoch - 65ms/step
Epoch 5/30
500/500 - 36s - loss: 0.3506 - accuracy: 0.8851 - 36s/epoch - 72ms/step
Epoch 6/30
500/500 - 35s - loss: 0.2903 - accuracy: 0.9046 - 35s/epoch - 70ms/step
Epoch 7/30
500/500 - 37s - loss: 0.2399 - accuracy: 0.9184 - 37s/epoch - 73ms/step
Epoch 8/30
500/500 - 36s - loss: 0.2116 - accuracy: 0.9304 - 36s/epoch - 72ms/step
Epoch 9/30
500/500 - 38s - loss: 0.1803 - accuracy: 0.9381 - 38s/epoch - 76ms/step
Epoch 10/30
500/500 - 29s - loss: 0.1580 - accuracy: 0.9476 - 29s/epoch - 59ms/step
Epoch 11/30
500/500 - 29s - loss: 0.1484 - accuracy: 0.9509 - 29s/epoch - 58ms/step
Epoch 12/30
500/500 - 25s - loss: 0.1325 - accuracy: 0.9543 - 25s/epoch - 50ms/step
E

## Save Model

In [2]:
#use the save() method
model.save("Text_emotion.h5")

NameError: ignored

## Test the model

In [3]:
#use the predict() method for model prediction
sentence = ["I am happy to meet my friends. We are planning to go to a party","I had a bad day at school. I got hurt while playing footlball"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences,maxlen = max_length,padding = padding_type,truncating = trunc_type)
result = model.predict(padded)
predict_class = np.argmax(result,axis = 1)
predict_class

NameError: ignored